# Inference - Large

This file evaluates the machine learning model on the full MNIST test set.  

See the bottom for the code you need to accelerate.  


In [2]:
%matplotlib inline
import cProfile
import json
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from scipy.stats import truncnorm
import timeit
import multiprocessing as mp
#from Network_Class import Network
import pickle

In [3]:
image_size = 28 # width and length
no_of_different_labels = 10 #  i.e. 0, 1, 2, 3, ..., 9
image_pixels = image_size * image_size
data_path = "./data/"

In [4]:
test_data = pd.read_csv(data_path + "mnist_test.csv", delimiter=",").values

fac = 0.99 / 255
test_imgs = np.asfarray(test_data[:, 1:], dtype=np.float32) * fac + 0.01
test_imgs = test_imgs.reshape(test_imgs.shape[0], 1, test_imgs.shape[1])

test_labels = np.asfarray(test_data[:, :1], dtype=np.float32)

lr = np.arange(no_of_different_labels)
# transform labels into one hot representation
test_labels_one_hot = (lr==test_labels).astype(np.float32)

# we don't want zeroes and ones in the labels neither:
test_labels_one_hot[test_labels_one_hot==0] = 0.001
test_labels_one_hot[test_labels_one_hot==1] = 0.999

## Machine Learning Model

Based on: https://towardsdatascience.com/math-neural-network-from-scratch-in-python-d6da9f29ce65

All training (backward propagation) code removed

In [5]:
# Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # backward_propagation removed

In [76]:
# inherit from base class Layer
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        #self.output = np.linalg.multi_dot([self.input, self.weights]) + self.bias
        return self.output

    # backward_propagation removed

In [77]:
# inherit from base class Layer
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.tanh(self.input)
        return self.output

    # backward_propagation removed

In [78]:
class TanhLayer(ActivationLayer):
    # static
    e = 2.71828182845904523536028747135266249775724709369995
    
    #http://www.plunk.org/~hatch/rightway.php
    #https://math.stackexchange.com/questions/518758/alternative-form-for-sinhx-coshx
    @staticmethod
    def tanh(x):
        e = TanhLayer.e
        #return np.tanh(x)
        #return (1-np.exp(-2 * x))/(1+np.exp(-2 * x))
        return (1 - e ** (-2 * x)) / (1 + e ** (-2 * x)) 

    @staticmethod
    def tanh_prime(x):
        return 1-TanhLayer.tanh(x)**2
    
    def __init__(self):
        super(TanhLayer,self).__init__(self.tanh, self.tanh_prime)

In [79]:
def mse(): pass
def mse_prime(): pass

In [80]:
class Network:
    def __init__(self):
        self.layers = []

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []
        #print("Layers: ", self.layers)
        # run network over all samples
        for i in range(samples):
            #print("HELLO?")
            # forward propagation
            output = input_data[i]
            #print("Layers:",self.layers)
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result
    
    # backward_propagation removed

    @classmethod
    def load(cls, fname):
        import pickle
        with open(fname, "br") as fh:
            return pickle.load(fh)

## Load the model

In [81]:
net = Network.load('network.pkl')

In [82]:
def evaluate_mp(net, data, labels, pipe):
    
    data_len = len(data)
    
    corrects,wrongs = 0,0
    for i in range(data_len):
        res = np.array(net.predict(data[i]))
        res = res.argmax()
        
        if res == labels[i]:
            corrects += 1
        else:
            wrongs += 1
    
    send_data = [corrects, wrongs]
    pipe.send(send_data)
    
    
def evaluate(net, data, labels, numProcesses):
    corrects, wrongs = 0, 0
    
    jobs = []
    
    recv_end, send_end = mp.Pipe()
    
    for i in range(numProcesses):
        jobs.append(mp.Process(target=evaluate_mp, args=(net, data[i], labels[i], send_end)))
        jobs[i].start()

    for i in range(numProcesses):
        recv = recv_end.recv()
        corrects += recv[0]
        wrongs += recv[1]
    send_end.close()
    recv_end.close()
    
    for proc in jobs:
        proc.join()
   
    
    return corrects, wrongs




In [88]:
numProcesses = 2

# Split the data and labels into numProcess arrays for each process to use
### i.e. for two processes, split the data into two arrays so each process take half of the entire data
# Limit of 4 processes for the pynq
data1 = []
data2 = []
data3 = []
data4 = []
data = [data1,data2,data3,data4]
labels1 = []
labels2 = []
labels3 = []
labels4 = []
labels = [labels1,labels2,labels3,labels4]

num = numProcesses
for i in range(num):
    data_index1 = int(len(test_imgs)/num)*num - ((num-i)*int(len(test_imgs)/num))
    data_index2 = int(len(test_imgs)/num) + i*int(len(test_imgs)/num)
    labels_index1 = int(len(test_labels)/num)*num - ((num-i)*int(len(test_labels)/num))
    labels_index2 = int(len(test_labels)/num) + i*int(len(test_labels)/num)

    if(i==num-1):
        data_index2+=1
        labels_index2+=1

    data[i] = test_imgs[data_index1:data_index2]
    labels[i] = test_labels[labels_index1:labels_index2]

# ============================
# This is the part you need to accelerate:
# ============================

In [89]:
start = timeit.default_timer()

corrects, wrongs = evaluate(net, data, labels, numProcesses)

stop = timeit.default_timer()

print ("Total Correct:" + str(corrects))
print ("Total Incorrect: " + str(wrongs))
print("Overall Accruracy: " + str(corrects / ( corrects + wrongs)))
print("Overall Accruracy (%): " + str( int( 1000* corrects / ( corrects + wrongs)) / 10) + "%")
print ()
print('Run Time: ' + str(stop - start) + ' Seconds')  

Total Correct:9565
Total Incorrect: 434
Overall Accruracy: 0.9565956595659566
Overall Accruracy (%): 95.6%

Run Time: 8.239653669297695 Seconds


### For reference, my Pynq takes ~22.4 seconds for the above evaluation.  

# Your mission:  Make this run faster, while keeping >95% overall accuracy!

### Please refer to the "Interence_Small" for details about swapping out individual layers

In [66]:
cProfile.run("evaluate(net,data,labels,numProcesses)")

         303 function calls in 8.959 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:416(parent)
        3    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:997(_handle_fromlist)
        1    0.001    0.001    8.958    8.958 <ipython-input-63-3ae8ffb840f8>:19(evaluate)
        1    0.000    0.000    8.958    8.958 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:38(_remove)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:81(add)
        2    0.000    0.000    0.000    0.000 connection.py:117(__init__)
        2    0.000    0.000    0.000    0.000 connection.py:130(__del__)
        2    0.000    0.000    0.000    0.000 connection.py:134(_check_closed)
        2    0.000    0.000    0.000    0.000 connection.py:138(_check_readable)
        2    0.000    0.000    0.000    0.000 connection.py